In [83]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
from datetime import datetime

In [84]:
# define the fonts to use for plots
family = 'Myriad Pro'
title_font = fm.FontProperties(family=family, style='normal', size=20, weight='normal', stretch='normal')
label_font = fm.FontProperties(family=family, style='normal', size=16, weight='normal', stretch='normal')
ticks_font = fm.FontProperties(family=family, style='normal', size=12, weight='normal', stretch='normal')

In [85]:
# load the csv data file
data = pd.read_csv('../resources/messages.csv')

In [86]:
# extracting exact datetimes based on the timestamps
all_dates = [*map(lambda ts : datetime.fromtimestamp(ts/1000).date(), data['timestamp'])]

In [87]:
# get the count per date
date_counts = pd.Series(all_dates).value_counts().sort_index()
print('There are {:,} dates with messages.'.format(len(date_counts)))
date_counts.head()

There are 2,659 dates with messages.


2011-03-12    60
2011-03-13    24
2011-03-14    17
2011-03-15     6
2011-03-16     1
dtype: int64

In [88]:
date_range = pd.date_range(start=min(all_dates), end=max(all_dates), freq='D')
index = date_range.map(lambda x: str(x.date()))
date_counts = date_counts.reindex(index, fill_value=0)
print('There are {:,} dates total in the range, with or without messages.'.format(len(date_counts)))
date_counts.head()

There are 3,344 dates total in the range, with or without messages.


2011-03-12    0
2011-03-13    0
2011-03-14    0
2011-03-15    0
2011-03-16    0
dtype: int64